Ball Positions Prediction in Field

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms,datasets
from copy import deepcopy as dc 
from sklearn.preprocessing import MinMaxScaler

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [21]:
df = pd.read_csv('../data/results/ball_positions_interpolated.csv')
df = df[['Frame','ball_x','ball_y']]

#df['ball_pos'] = list(zip(df['ball_x'], df['ball_y']))
#df = df[['Frame', 'ball_pos']]
df

,Frame,ball_x,ball_y
0,0,1624.50,492.00
1,1,1652.00,501.00
2,2,1655.00,496.00
3,3,1542.00,393.50
4,4,1540.00,392.50
...,...,...,...
153,153,116.50,518.50
154,154,106.50,507.50
155,155,98.25,497.25
156,156,90.00,487.00


In [23]:
#prepare data from df to lstm
def prepare_data_for_lstm(df,n_steps):
    df = dc(df)
    
    df.set_index('Frame', inplace=True)
    
    for i in range(1, n_steps + 1):
        df[f'ball_x_{i}_f-ago'] = df['ball_x'].shift(i)
        df[f'ball_y_{i}_f-ago'] = df['ball_y'].shift(i)
        
        #df[f'ball_x_{i}_frames_ago'] = df['ball_pos'].shift(i)
    
    df.dropna(inplace=True)
    return df 
    
lookback = 10
shifted_df = prepare_data_for_lstm(df,lookcback)
shifted_df_np = shifted_df.to_numpy()


In [24]:
scaler = MinMaxScaler(feature_range=(-1, 1))
shifted_df_np = scaler.fit_transform(shifted_df_np)